In [1]:
import pandas as pd


In [3]:
data_dengue = pd.read_csv('../../_data/data_dengue_peru.csv')
data_dengue

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_41280\61817505.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data_dengue = pd.read_csv('../../_data/data_dengue_peru.csv')


,Año,Semana,Eventos o daños,Departamento,Distrito,Provincia,Ubigeo,Casos
0,2020,43,Dengue,CUSCO,MEGANTONI,LA CONVENCION,80914,2.0
1,2021,41,Dengue,MOQUEGUA,MOQUEGUA,MARISCAL NIETO,180101,NaN
2,2021,42,Dengue,MOQUEGUA,MOQUEGUA,MARISCAL NIETO,180101,NaN
3,2020,1,Dengue,CUSCO,MEGANTONI,LA CONVENCION,80914,0.0
4,2020,53,Dengue,CUSCO,MEGANTONI,LA CONVENCION,80914,2.0
...,...,...,...,...,...,...,...,...
172139,2021,49,Dengue,MADRE DE DIOS,MADRE DE DIOS,MANU,170203,NaN
172140,2021,49,Dengue,MADRE DE DIOS,TAHUAMANU,TAHUAMANU,170303,NaN
172141,2021,50,Dengue,MADRE DE DIOS,MADRE DE DIOS,MANU,170203,NaN
172142,2021,50,Dengue,MADRE DE DIOS,TAHUAMANU,TAHUAMANU,170303,NaN


In [4]:

# Cargar la base de datos (reemplaza 'archivo.csv' con la ruta de tu archivo)
#df = pd.read_csv('archivo.csv')

# Generar el ubigeo de Departamentos (primeros 2 dígitos)
data_dengue['Ubigeo_Departamento'] = data_dengue['Ubigeo'].astype(str).str[:2]

# Generar el ubigeo de Provincias (primeros 4 dígitos)
data_dengue['Ubigeo_Provincia'] = data_dengue['Ubigeo'].astype(str).str[:4]

# Mostrar las primeras filas para verificar
print(data_dengue[['Ubigeo', 'Ubigeo_Departamento', 'Ubigeo_Provincia']].head())


   Ubigeo Ubigeo_Departamento Ubigeo_Provincia
0   80914                  80             8091
1  180101                  18             1801
2  180101                  18             1801
3   80914                  80             8091
4   80914                  80             8091
